In [27]:
import pandas as pd
import numpy as np

In [28]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [29]:
ratio_actividad = pd.read_csv(pathdata + 'data_output/indicadores/01_ratio_actividad.csv', index_col=0)
empleo_asalariado_con_descuento = pd.read_csv(pathdata + 'data_output/indicadores/02_empleo_asalariado_con_descuento.csv', index_col=0)
horas_remuneradas_media = pd.read_csv(pathdata + 'data_output/indicadores/03_horas_promedio_ocup_ppal.csv', index_col=0)
ingreso_salarial_media = pd.read_csv(pathdata + 'data_output/indicadores/04_ingreso_salarial.csv', index_col=0)
ratio_minutos_promedio_no_pago = pd.read_csv(pathdata + 'data_output/indicadores/05_tiempo_no_remunerado.csv', index_col=0)
inactivos_con_ingreso = pd.read_csv(pathdata + 'data_output/indicadores/06_dependencia_economica.csv', index_col=0)
hogares_jefatura = pd.read_csv(pathdata + 'data_output/indicadores/07_hogares_no-pobres_jefatura.csv', index_col=0)

In [74]:
ratio_actividad = ratio_actividad.sort_index()
empleo_asalariado_con_descuento = empleo_asalariado_con_descuento.sort_index()
horas_remuneradas_media = horas_remuneradas_media.sort_index()
ingreso_salarial_media = ingreso_salarial_media.sort_index()
ratio_minutos_promedio_no_pago = ratio_minutos_promedio_no_pago.sort_index()
inactivos_con_ingreso = inactivos_con_ingreso.sort_index()
hogares_jefatura = hogares_jefatura.sort_index()


In [75]:
variables = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, ingreso_salarial_media, ratio_minutos_promedio_no_pago, inactivos_con_ingreso, hogares_jefatura]

indices_ordenados = True

# Compara los índices de los DataFrames en un bucle
for i in range(len(variables) - 1):
    if not variables[i].index.equals(variables[i + 1].index):
        indices_ordenados = False
        break

# Imprime el resultado
print("Los índices están ordenados de la misma manera:", indices_ordenados)

Los índices están ordenados de la misma manera: True


In [76]:
empleo_asalariado_con_descuento['Formalidad'] = np.where(empleo_asalariado_con_descuento['Formalidad'] > 100, 100, empleo_asalariado_con_descuento['Formalidad'])


In [77]:
variables = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, ingreso_salarial_media, ratio_minutos_promedio_no_pago, inactivos_con_ingreso, hogares_jefatura]
columnas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio', 'Tiempo TNR', 'Dependencia', 'No Pobreza']

for columna, variable in zip(columnas,variables):
    
    variable['Ranking'] = variable[columna].rank(ascending=False, method='first')


In [78]:
componente_insercion = (ratio_actividad['Actividad'] + \
                        empleo_asalariado_con_descuento['Formalidad'] + \
                        horas_remuneradas_media['Horas promedio remuneradas'] + \
                        ingreso_salarial_media['Ingreso laboral promedio'])/4

componente_insercion = componente_insercion.to_frame()
componente_insercion.columns = ['Insercion laboral']
componente_insercion['Ranking'] = componente_insercion['Insercion laboral'].rank(ascending=False, method='first')

In [79]:
componente_autonomia_economica = (ratio_minutos_promedio_no_pago['Tiempo TNR'] + \
                                inactivos_con_ingreso['Dependencia'] + \
                                hogares_jefatura['No Pobreza'])/3

componente_autonomia_economica = componente_autonomia_economica.to_frame()
componente_autonomia_economica.columns = ['Autonomia economica']
componente_autonomia_economica['Ranking'] = componente_autonomia_economica['Autonomia economica'].rank(ascending=False, method='first')

In [83]:
indice_mti = np.sqrt(componente_autonomia_economica['Autonomia economica'] * componente_insercion['Insercion laboral']).to_frame()
indice_mti.columns = ['Indice MTeI']
indice_mti['Ranking'] = indice_mti['Indice MTeI'].rank(ascending=False, method='first')

In [36]:
## Genero tabla output

In [85]:
provincias = indice_mti.index.to_list()
provincia_repetidos = [value for value in provincias for _ in range(7)]

componentes = ['Insercion laboral'] * 4 + ['Autonomia Economica'] * 3
componentes_repetidos = componentes*24

indicadores = ['Actividad', 'Formalidad', 'Horas trabajadas', 'Ingreso salarial',
                  'Tiempo trabajo no remunerado', 'Dependencia economica', 'No-Pobreza']

In [86]:
data = {
    'Provincia': provincia_repetidos,
    'Componente': componentes_repetidos,
    'Indicador': indicadores*24
}

df = pd.DataFrame(data)

multiindex = pd.MultiIndex.from_frame(df[['Provincia', 'Componente', 'Indicador']])
df.set_index(multiindex, inplace=True)
df.drop(columns=['Provincia', 'Componente', 'Indicador'], inplace=True)

df[['Mujer', 'Varon', 'Ratio', 'N pob varon', 'N pob mujer', 'N pob tot', 'N varon', 'N mujer', 'CV', 'Error', 'Ranking indicador', 'Componentes', 'Ranking componente', 'Indice', 'Ranking Indice']] = pd.NaT

In [87]:
for indicador, columna, variable in zip(indicadores, columnas, variables):
    if indicador == 'Tiempo trabajo no remunerado':
        values = variable[['Mujer', 'Varon', columna, 'Ranking']].values
        df.loc[df.index.get_level_values('Indicador') == indicador, ['Mujer', 'Varon', 'Ratio', 'Ranking indicador']] = values

    else:    
        values = variable[['Mujer', 'Varon', columna, 'N_pob_v', 'N_pob_m', 'N_pob_tot', 'N_v', 'N_m', 'ER', 'ME', 'Ranking']].values
        df.loc[df.index.get_level_values('Indicador') == indicador, ['Mujer', 'Varon', 'Ratio', 'N pob varon', 'N pob mujer', 'N pob tot', 'N varon', 'N mujer', 'CV', 'Error', 'Ranking indicador']] = values


In [93]:
componente_insercion.to_csv(pathdata + 'data_output/componentes_e_indice/01_insercion_laboral.csv')
componente_autonomia_economica.to_csv(pathdata + 'data_output/componentes_e_indice/02_insercion_autonomia_economica.csv')
indice_mti.to_csv(pathdata + 'data_output/componentes_e_indice/01_indice_MTeI.csv')

In [98]:
df.to_csv(pathdata + 'data_output/tablas/01_indicadores_principales.csv')